In [1]:
import numpy as np
import pandas as pd
from matplotlib import font_manager, rc
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt

font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)
matplotlib.rcParams['axes.unicode_minus'] = False

In [2]:
data = pd.read_csv("C:/Users/korea/Desktop/문우혁/Github/BigContest_Flood-ZERO/문우혁/Data_Before/train.csv")
data.head(3)

,홍수사상번호,연,월,일,시간,유입량,유역평균강수1,강우A1,강우B1,강우C1,...,강우D5,수위E5,수위D5,유역평균강수6,강우A6,강우B6,강우C6,강우D6,수위E6,수위D6
0,1.0,2006.0,7.0,10.0,8.0,189.100000,6.4,7,7,7,...,8,2.54,122.660,6.4,7,7,8,8,2.54,122.61
1,1.0,2006.0,7.0,10.0,9.0,216.951962,6.3,7,8,7,...,10,2.53,122.648,7.3,7,8,10,10,2.53,122.60
2,1.0,2006.0,7.0,10.0,10.0,251.424419,6.4,7,9,7,...,11,2.53,122.636,8.2,7,9,10,11,2.53,122.59


In [3]:
X = data.drop('유입량', axis=1)
y = data['유입량']

In [4]:
X.head()

,홍수사상번호,연,월,일,시간,유역평균강수1,강우A1,강우B1,강우C1,강우D1,...,강우D5,수위E5,수위D5,유역평균강수6,강우A6,강우B6,강우C6,강우D6,수위E6,수위D6
0,1.0,2006.0,7.0,10.0,8.0,6.4,7,7,7,8,...,8,2.54,122.660,6.4,7,7,8,8,2.54,122.610
1,1.0,2006.0,7.0,10.0,9.0,6.3,7,8,7,8,...,10,2.53,122.648,7.3,7,8,10,10,2.53,122.600
2,1.0,2006.0,7.0,10.0,10.0,6.4,7,9,7,8,...,11,2.53,122.636,8.2,7,9,10,11,2.53,122.590
3,1.0,2006.0,7.0,10.0,11.0,7.3,7,10,7,8,...,14,2.53,122.620,11.3,9,10,15,14,2.53,122.585
4,1.0,2006.0,7.0,10.0,12.0,8.2,7,12,8,10,...,16,2.53,122.604,14.4,12,12,18,16,2.53,122.575


In [5]:
y.head()

0    189.100000
1    216.951962
2    251.424419
3    302.812199
4    384.783406
Name: 유입량, dtype: float64

In [6]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [20]:
rf = RandomForestRegressor(max_depth=6, min_samples_leaf=3, min_samples_split=3)
rf.fit(X_train, y_train)

RandomForestRegressor(max_depth=6, min_samples_leaf=3, min_samples_split=3)

In [21]:
predict = rf.predict(X_test)
expected = y_test

In [22]:
print('RMSE : {}'.format(mean_squared_error(expected, predict, squared=False)))

RMSE : 541.3648889925892


In [16]:
param_grid = {
    'n_estimators' : [10, 30, 50, 100, 200],
    'max_depth' : [6, 8, 10, 12],
    'min_samples_leaf' : [3, 5, 7, 10],
    'min_samples_split' : [2, 3, 5, 10]
}

In [18]:
rf_grid = GridSearchCV(rf, param_grid=param_grid, scoring='neg_mean_squared_error', n_jobs=-1, verbose=1)
rf_grid.fit(X_train, y_train)

Fitting 5 folds for each of 320 candidates, totalling 1600 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   12.2s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   45.8s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-1)]: Done 1600 out of 1600 | elapsed:  7.4min finished


GridSearchCV(estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'max_depth': [6, 8, 10, 12],
                         'min_samples_leaf': [3, 5, 7, 10],
                         'min_samples_split': [2, 3, 5, 10],
                         'n_estimators': [10, 30, 50, 100, 200]},
             scoring='neg_mean_squared_error', verbose=1)

In [27]:
print('점수 : {}'.format(np.sqrt(-rf_grid.best_score_)))
print('최적 파라미터 : {}'.format(rf_grid.best_params_))

점수 : 1077.6507091150727
최적 파라미터 : {'max_depth': 6, 'min_samples_leaf': 3, 'min_samples_split': 3, 'n_estimators': 100}
